In [ ]:
import pandas as pd
import numpy as np
from os.path import join, exists, basename
import argparse

In [ ]:
dir_in = 'path/to/input-data'
file_out = 'path/to/output-file'

In [ ]:
print(f'Input: {dir_in}')
print(f'Output: {file_out}')

In [ ]:
poss_files = [
    f'5_mast/mast.tsv',
    f'6_distinct/distinct.tsv',
    f'7_deseq2/deseq2.tsv',
    f'8_permutation/permutation.tsv',
    f'9_hierarchical-bootstrapping/hierarchical-bootstrapping_adv.tsv',
    f'10_scvi/scvi.tsv',
    f'11_dream/dream.tsv'
]
poss_files

In [ ]:
files = []
for file in poss_files:
    if exists(join(dir_in, file)):
        files.append(join(dir_in, file))
    else:
        print(f'{file} missing')
files

In [ ]:
pvalues = {}

for file in files:
    method = basename(file).split('.')[0].split('_')[0]
    if method == 'hierarchical-bootstrapping':
        res = pd.read_csv(file, sep='\t', index_col=0)['pvalue']
    elif method == 'permutation':
        res = pd.read_csv(file, sep='\t', index_col=0)['pvalue']
    elif method == 'deseq2':
        res = pd.read_csv(file, sep='\t', index_col=0)['pvalue']
    elif method == 'distinct':
        res = pd.read_csv(file, sep='\t', index_col=0)['p_val'].rename('pvalue')
        res.index.name = None
    elif method == 'mast':
        output = pd.read_csv(file, sep = '\t')
        pval = output[np.logical_and(output['component'] == 'H',output['contrast'] == 'Conditionstim')]
        logfc = output[np.logical_and(output['component'] == 'logFC', output['contrast'] == 'Conditionstim')]
        res = pd.merge(pval, logfc, on='primerid', suffixes=('_pval', '_logfc')).set_index('primerid')
        res = res['Pr(>Chisq)_pval'].rename('pvalue')
    elif method == 'scvi':
        res = pd.read_csv(file, sep = '\t', index_col=0)['proba_not_de']
    elif method == 'dream':
        res = pd.read_csv(file, sep = '\t', index_col=0)['P.Value']
    else:
        raise Exception('Wrong file', file)

    pvalues[method] = res


In [ ]:
results = pd.DataFrame.from_dict(pvalues)
results.to_csv(file_out, sep = "\t")